In [1]:
import pandas as pd
import numpy as np
from pathlib import Path  
import matplotlib.pyplot as plt

In [2]:
df_stock_price=pd.read_csv('./data/[ADJUSTED] SAP Dataset 2016-2021 - XETRA EUR.csv',na_values=['null'],index_col='Date',parse_dates=True,infer_datetime_format=True)
df_EURO_exchange_rate=pd.read_csv('./data/[ADJUSTED] EURO x BRL 2016-2021.csv',na_values=['null'],index_col='Date',parse_dates=True,infer_datetime_format=True)

In [3]:
print(df_stock_price.shape)
print(df_EURO_exchange_rate.shape)

(1519, 6)
(1519, 6)


In [4]:
#Adjust dataset with same date index
missing_dates_df1 = pd.date_range(start = '2016-01-01', end = '2021-12-31' ).difference(df_stock_price.index)
missing_dates_df2 = pd.date_range(start = '2016-01-01', end = '2021-12-31' ).difference(df_EURO_exchange_rate.index)

dates_missing1 = set(missing_dates_df1) - set(missing_dates_df2)
dates_missing2 = set(missing_dates_df2) - set(missing_dates_df1)
len(dates_missing1)

0

In [5]:
len(dates_missing2)

0

In [6]:
df_EURO_exchange_rate = df_EURO_exchange_rate.drop(dates_missing1)
df_EURO_exchange_rate.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1519 entries, 2016-01-04 to 2021-12-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1519 non-null   float64
 1   High       1519 non-null   float64
 2   Low        1519 non-null   float64
 3   Close      1519 non-null   float64
 4   Adj Close  1519 non-null   float64
 5   Volume     1519 non-null   float64
dtypes: float64(6)
memory usage: 83.1 KB


In [7]:
df_stock_price = df_stock_price.drop(dates_missing2)
df_stock_price.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1519 entries, 2016-01-04 to 2021-12-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1519 non-null   float64
 1   High       1519 non-null   float64
 2   Low        1519 non-null   float64
 3   Close      1519 non-null   float64
 4   Adj Close  1519 non-null   float64
 5   Volume     1519 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 83.1 KB


In [46]:
filepath = Path('./data/[ADJUSTED] SAP Dataset 2016-2021 - XETRA EUR.csv') 
df_stock_price.to_csv(filepath)

In [47]:
filepath = Path('./data/[ADJUSTED] EURO x BRL 2016-2021.csv') 
df_EURO_exchange_rate.to_csv(filepath)

In [18]:
merged_dataframe = pd.merge(df_stock_price["Close"], df_EURO_exchange_rate["Close"], on=["Date"])

merged_dataframe.columns = ['Close_Stock', 'Close_EUR']
merged_dataframe["Close_Stock(BRL)"] = merged_dataframe["Close_Stock"] * merged_dataframe["Close_EUR"]

merged_dataframe = merged_dataframe.rename(columns={'Close_Stock': 'Close_Stock(EUR)'})

merged_dataframe.head()

Close_Stock(EUR)  Close_EUR  Close_Stock(BRL)
Date                                                     
2016-01-04         70.580002     4.2891        302.724687
2016-01-05         71.430000     4.3710        312.220530
2016-01-06         72.050003     4.3301        311.983718
2016-01-07         71.339996     4.3322        309.059131
2016-01-08         71.050003     4.4145        313.650238

In [19]:
merged_dataframe.dropna()
filepath = Path('./data/Stock_Price_BRL 2016-2021.csv') 
merged_dataframe.to_csv(filepath)

In [21]:
ax = plt.gca()

merged_dataframe.plot(kind='line',y='Close_Stock(BRL)', ax=ax)
plt.xlabel('Ano')
plt.ylabel('Preço (BRL)')
plt.show()